In [19]:
## Libraries

import csv
from bs4 import BeautifulSoup
from selenium import webdriver

## Search Term

def get_url(search_term):
    """Generate a url from search term"""
    template = 'https://www.imdb.com/search/keyword/?keywords={}'
    search_term = search_term.replace(' ','-')
    
    # add term query to url
    url = template.format(search_term)
    
    # add page query placeholder
    url += '&page={}' 
    
    return url

## Extract Record

def extract_record(item):
    """Extract and return data from a single record"""
    
    # Title
    try:
        atag = item.h3.a
        description = atag.text.strip()
        Title = atag.get('href')
    except AttributeError:
        return
    
    # Genre
    try:
        Genre = item.find("span", class_="genre").text.rstrip().replace("\n","").split(",")
    except AttributeError:
        Genre = ''
        
    # Year
    try:
        Year = item.find("span", class_ = "lister-item-year text-muted unbold").text
    except AttributeError:
        Year = ''
            
    # Rating
    try:
        Rating_parent = item.find('strong').text
    except AttributeError:
        Rating_parent = ''
            
    result = (description, Genre, Year, Rating_parent)
    
    return result

## Main Function

def main(search_term):
    """Run main program routine"""
    # startup the webdriver
    driver = webdriver.Chrome()
    
    records = []
    url = get_url(search_term)
    
    for page in range(1,23):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'class': 'lister-item-content'})
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
                
    driver.close()
    
    #save data to csv file
    with open('youtube.csv', 'w', newline = '', encoding = 'utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Title', 'Genre', 'Year', 'Rating'])
        writer.writerows(records)

In [20]:
main('youtube')